In [1]:
import pandas as pd
from io import StringIO
import json
from pathlib import Path
from textwrap import dedent
import numpy as np
import psycopg2
import psycopg2.extras as extras
import pdb

_Helper methods_

In [11]:
def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [3]:
def json_to_df(_json):
    f = StringIO(_json)
    return pd.read_json(f)

# Start

In [4]:
connection = psycopg2.connect("dbname=hawc user=hawc")

fn = Path('data/transformed/writes.json')
writes = json.loads(fn.read_text())

## Study

In [8]:
_json = writes["study"]["study_study"]
df = json_to_df(_json)
execute_values(connection, df, "study_study")

Error: duplicate key value violates unique constraint "study_study_pkey"
DETAIL:  Key (reference_ptr_id)=(100864711) already exists.



1

## Animal

In [12]:
animal_order = [
    "animal_experiment",
    "animal_animalgroup", #"animal_animalgroup_parents",
    "assessment_baseendpoint",
    "animal_endpoint",
    "animal_endpointgroup"
]
if set(animal_order) != set(writes["animal"].keys()):
    raise Exception()

for table in animal_order:
    _json = writes["animal"][table]
    df = json_to_df(_json)
    if "created" in df.columns:
        df["created"] = df["created"].astype("datetime64[ms]")
    if "last_updated" in df.columns:
        df["last_updated"] = df["last_updated"].astype("datetime64[ms]")
    execute_values(connection, df, table)

Error: duplicate key value violates unique constraint "animal_experiment_pkey"
DETAIL:  Key (id)=(100500739) already exists.

Error: integer out of range

Error: duplicate key value violates unique constraint "assessment_baseendpoint_pkey"
DETAIL:  Key (id)=(227913) already exists.

Error: column "noel" of relation "animal_endpoint" does not exist
LINE 1: ...t_ptr_id,animal_group_id,response_units,data_type,NOEL,LOEL,...
                                                             ^

Error: smallint out of range

